#Converting the original metadata on the Books sample for Amazon reviews into a smaller DataFrame

The original metadata was a large 3.8GB file with more information than necessary - for the scope of our project - so we loaded into this notebook to change its format to a DataFrame and keep only the book titles, price, and ASIN codes we need to add to our existing reviews data.

In [28]:
import pandas as pd
import json
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#The original json file is in lines format - need the additional argument lines=True
book_titles = pd.read_json('/content/drive/MyDrive/BU_MSBA/BA820 - Unsupervised ML/meta_Books.json', lines=True)

In [12]:
book_titles.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [32]:
#Select the title, asin and price columns
books_meta = book_titles[['title', 'asin', 'price']]

In [34]:
#Missing values in price are marked as '', replace with NaN
books_meta.loc[:,'price'] = books_meta['price'].replace('', np.nan)

<ipython-input-34-739d00752f8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_meta.loc[:,'price'] = books_meta['price'].replace('', np.nan)


In [37]:
#Size of file was reduced from 3.8GB to 241MB
books_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934949 entries, 0 to 2934948
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   title   object
 1   asin    object
 2   price   object
dtypes: object(3)
memory usage: 67.2+ MB


In [38]:
#Save new dataframe
from google.colab import files

#Convert into csv
books_meta.to_csv('books_meta_short.csv')

#Download new csv file
files.download('books_meta_short.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>